<div style="text-align:center"><img src="zillowprojlogo.png"/></div>

<a id='navigation'></a>

<button class="button-save large">[Scenario](#scenario)</button>
<button class="button-save large">[Project Planning](#project-planning)</button>
<button class="button-save large">[Key Findings](#key-findings)</button>
<button class="button-save large">[Tested Hypotheses](#tested-hypotheses)</button>
<button class="button-save large">[Take Aways](#take-aways)</button>
<button class="button-save large">[Data Dictionary](#data-dictionary)</button>
<button class="button-save large">[Workflow](#workflow)</button>

<div class="alert alert-block alert-info"><a name="scenario"></a><h1><i class="fas fa-home"></i> Scenario</h1></div>
Selling homes in our new normal has just gotten easier with Zillow Offers®. Now home owners can hand over the burden of selling their property, by selling directly to us based on our state of the art Zestimate score.

The accuracy and integrity of our Zestimate score is of high importance. As a junior data scientists on the Zillow data science team, we are tasked with uncovering what drivers most affect the validity of the Zestimate score. This is measured by our target variable: `logerror`, which is the difference between Zillow's estimated Zestimate and actual sale price. 
>`logerror` = log (Zestimate) − log (ActualSalePrice)

### Project Goal: 
The goal for this project is to create a model that will accurately predict the Zestimate’s `logerror`. By doing so, we will uncover what features available on the Zillow Dataset are driving the amount of error.


In [1]:
# Imports needed

import pandas as pd
import numpy as np
import os
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from env import host, user, password
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.cluster import KMeans

import seaborn as sns
import matplotlib.pyplot as plt
import wrangle

# modeling methods
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor

## Acquire the Data

In [2]:
# Acquire the data from SQL

df = wrangle.zillow17()
df

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1087254,10711855,NaN,NaN,NaN,2.0,3.0,NaN,8.0,2.0,...,6.037113e+13,-0.007357,2017-07-07,None,None,None,Central,Single Family Residential,None,None
1,1072280,10711877,1.0,NaN,NaN,2.0,4.0,NaN,8.0,2.0,...,6.037113e+13,0.021066,2017-08-29,Central,None,None,Central,Single Family Residential,None,None
2,1340933,10711888,1.0,NaN,NaN,2.0,4.0,NaN,8.0,2.0,...,6.037113e+13,0.077174,2017-04-04,Central,None,None,Central,Single Family Residential,None,None
3,1878109,10711910,NaN,NaN,NaN,2.0,3.0,NaN,8.0,2.0,...,6.037113e+13,-0.041238,2017-03-17,None,None,None,Central,Single Family Residential,None,None
4,2190858,10711923,NaN,NaN,NaN,2.0,4.0,NaN,8.0,2.0,...,6.037113e+13,-0.009496,2017-03-24,None,None,None,Central,Single Family Residential,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77569,775695,167686999,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,-0.068632,2017-02-28,None,None,None,None,Single Family Residential,None,None
77570,2863262,167687739,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,0.360020,2017-03-03,None,None,None,None,Condominium,None,None
77571,1372384,167687839,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,0.038797,2017-05-31,None,None,None,None,Single Family Residential,None,None
77572,2758757,167688532,1.0,NaN,NaN,3.0,3.0,NaN,4.0,3.0,...,NaN,0.006706,2017-02-03,Central,None,None,Central,Condominium,None,None


### Acquire Takeaways:
- Massive dataset
- Many columns are primarily nulls/NaNs
- Many columns are redundant

## Prep the Data

In [15]:
# Dropping unnecessary/redundant columns, dropping nulls/NaNs, and  limiting outliers 

df, train, validate, test = wrangle.wrangle_zillow()

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71818 entries, 10711855 to 162960814
Data columns (total 23 columns):
bathrooms                 71818 non-null float64
bedrooms                  71818 non-null int64
property_quality          71818 non-null int64
sqft                      71818 non-null float64
fips                      71818 non-null int64
latitude                  71818 non-null float64
longitude                 71818 non-null float64
lot_sqft                  71818 non-null float64
rawcensustractandblock    71818 non-null float64
regionidcity              71818 non-null float64
zip_code                  71818 non-null int64
roomcnt                   71818 non-null int64
unitcnt                   71818 non-null int64
yearbuilt                 71818 non-null int64
structure_value           71818 non-null float64
home_value                71818 non-null float64
land_value                71818 non-null float64
taxamount                 71818 non-null float64
logerror    

In [14]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
bathrooms,71818.0,2.272822e+00,0.916359,1.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,1.000000e+01
bedrooms,71818.0,3.016765e+00,0.990155,1.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,1.100000e+01
property_quality,71818.0,6.758431e+00,1.350214,1.000000e+00,6.000000e+00,7.000000e+00,7.000000e+00,1.200000e+01
sqft,71818.0,1.741974e+03,873.320788,5.010000e+02,1.172000e+03,1.517000e+03,2.056000e+03,1.203900e+04
fips,71818.0,6.049319e+03,21.083111,6.037000e+03,6.037000e+03,6.037000e+03,6.059000e+03,6.111000e+03
latitude,71818.0,3.400713e+07,267350.751880,3.333953e+07,3.381053e+07,3.402124e+07,3.417671e+07,3.481877e+07
longitude,71818.0,-1.182004e+08,364289.055844,-1.194753e+08,-1.184182e+08,-1.181685e+08,-1.179179e+08,-1.175723e+08
lot_sqft,71818.0,2.775619e+04,118484.322103,2.360000e+02,5.985000e+03,7.265000e+03,1.061925e+04,6.971010e+06
rawcensustractandblock,71818.0,6.049622e+07,209253.579521,6.037101e+07,6.037400e+07,6.037621e+07,6.059052e+07,6.111009e+07
regionidcity,71818.0,3.350759e+04,45986.851085,3.491000e+03,1.244700e+04,2.521800e+04,4.545700e+04,3.965560e+05


### Data Prep Takeaways:
- Dropped unnecessary/redundant columns: 'id', 'calculatedbathnbr', 'finishedsquarefeet12', 'fullbathcnt', 'heatingorsystemtypeid',                 'propertycountylandusecode', 'propertylandusetypeid','propertyzoningdesc', 'censustractandblock', 'propertylandusedesc', 
  'heatingorsystemdesc', 'assessmentyear', 'regionidcounty'  
- Set our outliers to taxvaluedollarcnt < 5_000_000 and calculatedfinishedsquarefeet < 12500
- Dropped columns that had >60% nulls, dropped rows that had >70% nulls
- Dropped a total of 45 columns and 5756 rows
- Still a large dataset

## Explore the data

In [19]:
# Check our train, validate,test

train.shape, validate.shape, test.shape

((40217, 23), (17237, 23), (14364, 23))